## Single run as video

In [ ]:
# ==========================================
# 1. GLOBAL CONSTANTS & CONFIGURATION
# ==========================================
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
from matplotlib.animation import FuncAnimation
from IPython.display import HTML, clear_output
import sys
import time

# --- Numba Check ---
try:
    from numba import njit, prange
    USE_NUMBA = True
    print("Numba loaded successfully. Hardware acceleration enabled.")
except ImportError:
    USE_NUMBA = False
    print("Numba not found. Falling back to standard NumPy.")
    
    def njit(*args, **kwargs):
        def decorator(func): return func
        return args[0] if (len(args) == 1 and callable(args[0])) else decorator

    def prange(start, stop=None, step=1):
        return range(start, stop if stop is not None else start, step)

# Physics & Environment
affection_distance = 6
fuel_loss_rate = 0.1             # Used to be magic 0.1
ambient_temperature = 25.0
passive_cooling_coefficient = 0.05
drying_constant = 0.01 
ignition_threshold_dryness = 0.6 # Used to be magic 0.6
dt = 0.1
time_limit = 400
wind = np.array([0, 0]) 
wind_change_chance = 0.002 
shape = (100, 100)
starting_burning_amount = 2
numpy_seed = 5346
frames_to_skip = int(1/dt)

# Visuals
COLOR_GROUND = [0.78, 0.78, 0.78]
COLOR_FIRE_ORANGE = [1.0, 0.27, 0.0]
COLOR_CHARCOAL = [0.12, 0.12, 0.12]

# ==========================================
# 2. CORE PHYSICS ENGINE
# ==========================================

@njit(parallel=True)
def compute_avg_temp(average_temperature_grid, temp, wind_vec, dist):
    rows, cols = temp.shape
    side = 2 * dist + 1
    num_neighbors = float(side * side - 1)
    for i in prange(rows):
        for j in range(cols):
            total = 0.0
            for oi in range(-dist, dist + 1):
                ni = i + oi - wind_vec[0]
                for oj in range(-dist, dist + 1):
                    nj = j + oj - wind_vec[1]
                    if 0 <= ni < rows and 0 <= nj < cols:
                        if not (oi == 0 and oj == 0):
                            total += temp[ni, nj]
            average_temperature_grid[i, j] = total / num_neighbors

@njit(parallel=True)
def engine_step(temp, fuel, dryness, burning, 
                next_temp, next_fuel, next_dryness, next_burning,
                ground, c_temp, t_cond, e_dens, avg_grid, wind_vec, dt):
    rows, cols = temp.shape
    compute_avg_temp(avg_grid, temp, wind_vec, affection_distance) 
    for i in prange(rows):
        for j in range(cols):
            if ground[i, j]:
                next_temp[i,j], next_fuel[i,j], next_dryness[i,j], next_burning[i,j] = temp[i,j], fuel[i,j], dryness[i,j], burning[i,j]
                continue
            
            curr_f, curr_t, curr_b = fuel[i, j], temp[i, j], burning[i, j]
            
            if curr_b:
                f_loss = fuel_loss_rate * curr_f * dt
                if curr_f < 0.1: f_loss = curr_f
                next_fuel[i, j] = curr_f - f_loss
                next_temp[i, j] = curr_t + f_loss * e_dens[i, j]
                next_burning[i, j] = next_fuel[i, j] > 0
                next_dryness[i, j] = dryness[i, j]
            else:
                avg_t = avg_grid[i, j]
                # Heat transfer and passive cooling
                t_delta = (avg_t - curr_t) * t_cond[i, j] - (curr_t - ambient_temperature) * passive_cooling_coefficient
                next_temp[i, j] = curr_t + t_delta * dt
                
                # Evaporation/Drying
                d_inc = drying_constant * max(0.0, next_temp[i, j] - ambient_temperature) * dt
                next_dryness[i, j] = min(1.0, dryness[i, j] + d_inc)
                next_fuel[i, j] = curr_f
                
                # Ignition Logic
                burning_neighbors = 0
                for ni in range(i-1, i+2):
                    for nj in range(j-1, j+2):
                        if 0 <= ni < rows and 0 <= nj < cols and burning[ni, nj]:
                            burning_neighbors += 1
                
                risk = (next_temp[i, j] / c_temp[i, j]) * next_dryness[i, j]
                if (next_dryness[i, j] >= ignition_threshold_dryness or burning_neighbors > 1) and curr_f > 0:
                    if np.random.random() < risk * burning_neighbors * dt:
                        next_burning[i, j] = True
                        next_temp[i, j] = c_temp[i, j]
                    else:
                        next_burning[i, j] = False
                else:
                    next_burning[i, j] = False

# ==========================================
# 3. UTILITIES & VISUALIZATION
# ==========================================

def print_progress(current_iteration, max_iterations, display_eta=True, eta_update_rate=3):
    # 1. Initialize start time and a persistent ETA string
    if not hasattr(print_progress, "start_seconds") or current_iteration == 0:
        print_progress.start_seconds = time.time()
        print_progress.last_eta = "Calculating..."

    # 2. Calculate progress
    progress = max(current_iteration / max_iterations, 0.0001)
    
    # 3. Only update the ETA string every few frames to save resources and prevent jumping
    if display_eta and (current_iteration % eta_update_rate == 0 or current_iteration == 1):
        elapsed = time.time() - print_progress.start_seconds
        estimated_total = elapsed / progress
        print_progress.last_eta = f"{estimated_total:.2f}s"
    
    # 4. Construct a fixed-length output
    # We always include the last_eta to keep the string length stable
    output = f"{int(100 * progress):>3}% | Estimated total time: {print_progress.last_eta}"
    
    # 5. Write to stdout with carriage return and padding
    # .ljust(80) ensures any previous long lines are fully overwritten
    sys.stdout.write(f"\r{output.ljust(80)}")
    sys.stdout.flush()


def get_color_grid_from_arrays(c_temp_arr, temp_arr, fuel_arr, start_fuel_arr, b_arr, ground_arr, base_colors_grid):
    rows, cols = temp_arr.shape
    img = np.zeros((rows, cols, 3), dtype=np.float32)
    img[ground_arr] = COLOR_GROUND
    
    # Living vegetation (green to red heat glow)
    living_mask = (~b_arr) & (~ground_arr) & (fuel_arr > 0)
    ratio = np.clip((temp_arr - ambient_temperature) / (c_temp_arr - ambient_temperature), 0, 1) ** 0.5
    for c in range(3):
        img[living_mask, c] = base_colors_grid[living_mask, c] * (1 - ratio[living_mask]) + ([1.0, 0.0, 0.0][c] * ratio[living_mask])
    
    # Burning cells (charcoal to orange flame)
    ratio_calc = np.zeros_like(fuel_arr)
    np.divide(fuel_arr, start_fuel_arr, out=ratio_calc, where=(start_fuel_arr > 0))
    burn_ratio = np.clip(ratio_calc, 0, 1) ** 0.5
    for c in range(3):
        img[b_arr, c] = np.array(COLOR_CHARCOAL[c]) * (1 - burn_ratio[b_arr]) + np.array(COLOR_FIRE_ORANGE[c]) * burn_ratio[b_arr]
    
    # Dead/Burnt out
    img[(~b_arr) & (~ground_arr) & (fuel_arr <= 0)] = [0.0, 0.0, 0.0]
    return img

class Cell:
    def __init__(self, combustion_temp, thermal_conductivity, dryness, fuel_amount, energy_density, is_ground, rng_generator, base_color):
        self.combustion_temp, self.thermal_conductivity, self.dryness = combustion_temp, thermal_conductivity, dryness
        self.fuel_amount, self.starting_fuel_amount, self.energy_density = fuel_amount, fuel_amount, energy_density
        self.is_ground, self.is_burning, self.current_temperature = is_ground, False, ambient_temperature
        self.rng_generator, self.base_color = rng_generator, base_color
    def clone(self):
        c = Cell(self.combustion_temp, self.thermal_conductivity, self.dryness, self.fuel_amount, self.energy_density, self.is_ground, self.rng_generator, self.base_color)
        c.is_burning, c.current_temperature = self.is_burning, self.current_temperature
        return c
    def ignite(self):
        if self.fuel_amount > 0 and not self.is_ground:
            self.is_burning, self.current_temperature = True, self.combustion_temp
    def is_flammable(self):
        return self.fuel_amount > 0 and not self.is_ground

# ==========================================
# 4. INITIALIZATION
# ==========================================

rng_seeded = np.random.default_rng(numpy_seed)
cell_types = [
    Cell(
        300,            # combustion temperature
        0.05,           # thermal conductivity
        0.1,            # dryness
        500,             # starting fuel amount (proportional to mass)
        2,              # energy density (how much of spent fuel is converted to heat)
        False,          # is ground (can burn)
        rng_seeded,     # rng generator
        [46, 156, 6]    # base tree color (so we can differentiate)
        ),
    Cell(
        430,            # combustion temperature
        0.1,            # thermal conductivity
        0.0,            # dryness
        400,             # starting fuel amount (proportional to mass)
        3,              # energy density (how much of spent fuel is converted to heat)
        False,          # is ground (can burn)
        rng_seeded,     # rng generator
        [43, 69, 14]    # base tree color (so we can differentiate)
        ),
    Cell(
        100_000,            # combustion temperature
        0.001,            # thermal conductivity
        0.0,            # dryness
        0,             # starting fuel amount (proportional to mass)
        0,              # energy density (how much of spent fuel is converted to heat)
        True,          # is ground (can burn)
        rng_seeded,     # rng generator
        [171, 111, 51]    # base tree color (so we can differentiate)
        )
]
indices = rng_seeded.choice(len(cell_types), size=shape, p=[0.4, 0.4, 0.2])
forest_original = np.array([[cell_types[idx].clone() for idx in row] for row in indices])

for _ in range(starting_burning_amount):
    ri, rj = rng_seeded.integers(0, shape[0]), rng_seeded.integers(0, shape[1])
    while not forest_original[ri][rj].is_flammable():
        ri, rj = rng_seeded.integers(0, shape[0]), rng_seeded.integers(0, shape[1])
    forest_original[ri][rj].ignite()

def bake_forest(forest_objects):
    rows, cols = forest_objects.shape
    c_temp, t_cond, e_dens, s_fuel, ground, colors = [np.zeros((rows, cols), dtype=np.float32) for _ in range(4)] + [np.zeros((rows, cols), dtype=np.bool_), np.zeros((rows, cols, 3), dtype=np.float32)]
    temp, fuel, dry, burn = [np.zeros((rows, cols), dtype=np.float32) for _ in range(3)] + [np.zeros((rows, cols), dtype=np.bool_)]
    for i in range(rows):
        for j in range(cols):
            c = forest_objects[i, j]
            c_temp[i,j], t_cond[i,j], e_dens[i,j], s_fuel[i,j] = c.combustion_temp, c.thermal_conductivity, c.energy_density, c.starting_fuel_amount
            ground[i,j], colors[i,j] = c.is_ground, np.array(c.base_color)/255.0
            temp[i,j], fuel[i,j], dry[i,j], burn[i,j] = c.current_temperature, c.fuel_amount, c.dryness, c.is_burning
    return (c_temp, t_cond, e_dens, s_fuel, ground, colors, temp, fuel, dry, burn)

grids = bake_forest(forest_original)
(combustion_temperature_arr, thermal_conductivity_cond_arr, energy_density_arr, start_fuel_arr, is_ground_arr, base_colors_arr,
 current_temperature_arr, fuel_arr, dryness_arr, is_burning_arr) = grids

current_state = [current_temperature_arr, fuel_arr, dryness_arr, is_burning_arr]
next_state = [arr.copy() for arr in current_state]
avg_buf = np.zeros(shape, dtype=np.float32)

time_values, avg_temp_history = [], []

# ==========================================
# 5. ANIMATION & PLOTTING
# ==========================================

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))
matplotlib.rcParams['animation.embed_limit'] = 500.0

im = ax1.imshow(get_color_grid_from_arrays(combustion_temperature_arr, current_state[0], current_state[1], start_fuel_arr, current_state[3], is_ground_arr, base_colors_arr), animated=True)
ax1.set_axis_off()

temp_line, = ax2.plot([], [], color='red', lw=2)
ax2.set_xlim(0, time_limit)
ax2.set_ylim(20, 100)
ax2.set_title("Total Average Temperature")
ax2.set_xlabel("Time (ticks)")
ax2.set_ylabel("Temp (°C)")
ax2.grid(True)

def advance(index, write_progress=False):
    global current_state, next_state, wind
    if write_progress: print_progress(index, int(time_limit/dt))
    
    # --- Dynamic Wind Logic ---
    if np.random.random() < wind_change_chance:
        component = np.random.choice([0, 1]) # Y or X
        shift = np.random.choice([-1, 1])
        wind[component] = np.clip(wind[component] + shift, -5, 5) # Keep wind within reasonable bounds

    t_curr, f_curr, d_curr, b_curr = current_state
    t_next, f_next, d_next, b_next = next_state
    
    engine_step(t_curr, f_curr, d_curr, b_curr, t_next, f_next, d_next, b_next,
                is_ground_arr, combustion_temperature_arr, thermal_conductivity_cond_arr, 
                energy_density_arr, avg_buf, wind, dt)
    
    # Swap buffers
    current_state[0][:], current_state[1][:], current_state[2][:], current_state[3][:] = t_next, f_next, d_next, b_next

def anim_init():
    temp_line.set_data([], [])
    return (im, temp_line)

def anim_advance(frame):
    current_time = frame * dt
    for _ in range(frames_to_skip):
        advance(frame, True)
    
    # Update View
    new_colors = get_color_grid_from_arrays(combustion_temperature_arr, current_state[0], current_state[1], start_fuel_arr, current_state[3], is_ground_arr, base_colors_arr)
    im.set_array(new_colors)
    ax1.set_title(f"Time: {current_time:.1f} | Wind: [{wind[1]}, {wind[0]}] (X, Y)")

    # Update Graph
    avg_t = np.mean(current_state[0])
    time_values.append(current_time)
    avg_temp_history.append(avg_t)
    temp_line.set_data(time_values, avg_temp_history)
    
    if avg_t > ax2.get_ylim()[1]:
        ax2.set_ylim(20, avg_t * 1.2)
    
    return (im, temp_line)

anim = FuncAnimation(
    fig, anim_advance, init_func=anim_init,
    frames=range(0, int(time_limit/dt), frames_to_skip),
    interval=30, blit=False
)

HTML(anim.to_jshtml())